In [1]:
# 18-09-2024 The code below is a combination of already existing code by R. Buring, and adjustments made by F. Schipper 
# (particularly, the biggest adjustments are checking that the basis element(s) of the cocycle space are Hamiltonian, 
# commentary, printing of output and rewriting some code to be able to run it parallelized). It is supplementary material 
# accompanying proceedings written for the ISQS28 (Integrable Systems and Quantum Symmetries) conference (see also: arxiv link??).

# Extra packages that are useful
# for Pool, note that the # of processors throughout this is set to 24. Depending on your machine, you want to change this 
# amount. A larger problem in probably the amount of memory you have available. You can split up parts of the code, and run
# smaller amounts, while saving necessary vector fields, lists of encodings etc on disc (eg via the pickle library). Once you
# need the saved information, load it back in. On a 32GB RAM machine, the code ran after splitting it into 6 smaller pieces
from multiprocessing import Pool
import itertools

# We import the following (see https://github.com/rburing/gcaops) to be able to run the code.
from gcaops.graph.formality_graph import FormalityGraph
from gcaops.algebra.differential_polynomial_ring import DifferentialPolynomialRing
from gcaops.algebra.superfunction_algebra import SuperfunctionAlgebra
from gcaops.graph.undirected_graph_complex import UndirectedGraphComplex
from gcaops.graph.directed_graph_complex import DirectedGraphComplex

# We start by generating the encodings for the descendants of Gamma_11^2D and Gamma_12^2D
X_graph_encodings = []
for (i2,j1,j2,k) in itertools.product([2,5,8],[1,4,7],[1,4,7],[3,6,9]):
    X_graph_encodings.append((0,1,4,7,j1,k,5,8,j2,i2,6,9))

for (i1,i2,j1,j2,k) in itertools.product([2,5,8],[2,5,8], [1,4,7], [1,4,7], [3,6,9]):
    X_graph_encodings.append((0,i1,4,7,j1,k,5,8,j2,i2,6,9))

# To move from the encodings of the graphs to actual graphs, we use the next function. The function splits the 
# encoding by vertex via ;, and then the targets by ,. A graph is returned on 3 Levi-Civita vertices, 3 corresponding a^1 
# Casimir vertices, 3 corresponding a^2 Casimir vertices, 1 sink, and with edges (original vertex, target vertex). 
def encoding_to_graph(encoding):
    targets = [encoding[0:4], encoding[4:8], encoding[8:12]]
    edges = sum([[(k+1,v) for v in t] for (k,t) in enumerate(targets)], [])
    return FormalityGraph(1, 9, edges)

# The computation below still goes reasonably fast as there's only 324 graphs. When working with significantly more graphs
# (for instance with gamma_5 rather than gamma_3) you will want to parallelize these computations as well.
X_graphs = [encoding_to_graph(e) for e in X_graph_encodings]
print('We have', len(X_graphs), 'graphs.\n')

# Below, we check the how many graphs that were created via the encodings are (non)isomorphic. It does so in the following way:
# for each formality graph in X_graphs, it computes the edges of the canonical form (canonical in the sense that isomorphic
# graphs return the same canonical form) of said formality graph. If a graph with these edges already appears in X_graphs_iso,
# the new (isomorphic) graph is not added to the list. If the graph does not yet appear, it is added to the list. The list is
# indexed by h, NOT by an index from 0,...,40. For this, unmute the line X_graphs_iso=list(X_graphs_iso.values())
#X_graphs_iso={}
#for g in X_graphs:
#    h=tuple(g.canonical_form().edges())
#    if not h in X_graphs_iso:
#        X_graphs_iso[h]=g
#X_graphs_iso=list(X_graphs_iso.values())
#print ('There are', len(X_graphs_iso), 'nonisomorphic graphs.\n')

# Create the differential polynomial ring. We are working in 4D, so we have even coordinates x,y,z,w and the corresponding 
# odd coordinates xi[0], xi[1], xi[2] and xi[3]. rho is exactly the rho in a 4D Nambu-determinant Poisson bracket,  
# (P(f,g)= rho d(f,g,a^1,a^2)/d(x,y,z,w)), and a^1, a^2 are the Casimirs.  Finally, max_differential_orders  tells the 
# programme how many times rho and a^1/a^2 can be  differentiated. The maximum is stipulated by the graphs (we cannot have 
# double edges). Thus, the maximum in degree of each rho vertex is 4. As we will be looking at [[P,X]], we add an extra +1 to 
# the differential orders since taking the Schouten bracket with P introduces an extra derivative.
D4 = DifferentialPolynomialRing(QQ, ('rho','a1', 'a2'), ('x','y','z', 'w'), max_differential_orders=[3+1,4+1,4+1]) 
rho, a1, a2 = D4.fibre_variables()
x,y,z,w = D4.base_variables()
even_coords = [x,y,z,w]

S4.<xi0,xi1,xi2,xi3> = SuperfunctionAlgebra(D4, D4.base_variables()) 
xi = S4.gens()
odd_coords = xi

# We now compute the vector fields corresponding to the graphs. E is the Euler vector field in the sink (vertex 0), and
# epsilon is the Levi-Civita tensor. Note that we have a Levi-Civita tensor at each of the vertices 1, 2, 3. We first compute
# the sign of each term appearing in the formulas, and then compute the differential polynomial. Note that compared to 2D and 
# 3D, this is now a definition rather than a for loop; this is to be able to parallelize the computations.
epsilon = xi[0]*xi[1]*xi[2]*xi[3]
E = x*xi[0] + y*xi[1] + z*xi[2] + w*xi[3]
def evaluate_graph(g): 
    result = S4.zero()
    for index_choice in itertools.product(itertools.permutations(range(4)), repeat=3):
        sign = epsilon[index_choice[0]] * epsilon[index_choice[1]] * epsilon[index_choice[2]]
        vertex_content = [E, S4(rho), S4(rho), S4(rho), S4(a1), S4(a1), S4(a1), S4(a2), S4(a2), S4(a2)]
        for ((source, target), index) in zip(g.edges(), sum(map(list, index_choice), [])):
            vertex_content[target] = vertex_content[target].derivative(even_coords[index])
        result += sign * prod(vertex_content)
    return result

# We compute the formulas.
X_vector_fields = []
with Pool(processes=24) as pool:
    X_vector_fields = list(pool.imap(evaluate_graph, X_graphs))

# We check how many (if any) graphs evaluate to 0. 
zeros=X_vector_fields.count(0)
print('There are', zeros, 'graphs that evaluate to 0 under the morhpism from graphs to multivectors.')

# In case the are graphs that evaluate to 0, the line below shows which graphs do so (note that the counting starts from 1!).
print('These graphs are:')
for (k,X) in enumerate(X_vector_fields):
    if X==0:
        print('graph', k+1,)
print()

# The next part is to find out linear relations of the vector fields we just computed. We look at the monomials that appear
# in each xi[0], xi[1], xi[2] and xi[3] part of the vector fields, and store them in X_monomial_basis. 
X_monomial_basis = [set([]) for i in range(4)]
for i in range(4):
    for X in X_vector_fields:
        X_monomial_basis[i] |= set(X[i].monomials())
X_monomial_basis = [list(b) for b in X_monomial_basis]
X_monomial_count = sum(len(b) for b in X_monomial_basis)

# Next, we use this monomial basis to create a matrix that identifies each vector field by the monomials that appear in it.
X_monomial_to_index = [{monomial : idx for (idx,monomial) in enumerate(X_monomial_basis[j])} for j in range(4)] 
X_evaluation_matrix = matrix(QQ, X_monomial_count, len(X_vector_fields))
for i in range(len(X_vector_fields)):
    v = vector(QQ, X_monomial_count)
    index_shift = 0
    for j in range(4):
        f = X_vector_fields[i][j]
        for coeff, monomial in zip(f.coefficients(), f.monomials()):
            monomial_index = X_monomial_to_index[j][monomial]
            v[index_shift + monomial_index] = coeff
        index_shift += len(X_monomial_basis[j])
    X_evaluation_matrix.set_column(i, v)

# We can now detect linear relations by computing the nullity of this matrix. 
nullity = X_evaluation_matrix.right_nullity()
print('The vector fields have', nullity, 'linear relations among themselves.\n')


# Unmute the next line if you want to explicitly see the 28 linear relations among the vector fields.
#print('These relations are expressed by \n', X_evaluation_matrix.right_kernel().basis(), '\n')

# We save a list of linearly independent vector fields.
pivots = X_evaluation_matrix.pivots()
print(' A maximal subset of linearly independent graphs is given by:', list(pivots),'\n')

# Next, let us create the swapped encodings in order to be able skew-symmetrize the vector fields (skew-symmetrizing is with 
# respect to a^1 and a^2).
swapped_X_graph_encodings= []
for i in range(len(X_graph_encodings)):
    new_encoding=[]
    for j in range(12):
        if X_graph_encodings[i][j]==4:
             new_encoding.append(7)
        elif X_graph_encodings[i][j]==7:
            new_encoding.append(4)
        elif X_graph_encodings[i][j]==5:
            new_encoding.append(8)
        elif X_graph_encodings[i][j]==8:
            new_encoding.append(5)
        elif X_graph_encodings[i][j]==6:
            new_encoding.append(9)
        elif X_graph_encodings[i][j]==9:
            new_encoding.append(6)
        else:
            new_encoding.append(X_graph_encodings[i][j])
    swapped_X_graph_encodings.append(new_encoding)

# Now, we compute the formulas corresponding to the swapped encodings.
swapped_graphs = [encoding_to_graph(e) for e in swapped_X_graph_encodings]

# We find the corresponding vector fields.
swapped_vector_fields = []
with Pool(processes=24) as pool:
    swapped_vector_fields = list(pool.imap(evaluate_graph, swapped_graphs))

# We create skew vector fields from the linearly independent non-skewed vector fields. 
skew_vector_fields=[]
for i in pivots:
    skew_vector_fields.append(1/2*(X_vector_fields[i]-swapped_vector_fields[i]))

# We also create a new list storing the corresponding encodings (only the original encodings, not also the swapped encodings).
independent_encodings=[]
for i in pivots:
    independent_encodings.append(X_graph_encodings[i])

# While we got rid of linear dependencies in X_vector_fields, after skewing the vector fields, new dependencies show up. We
# again get rid of these in the exact same manner. 
X_skew_monomial_basis = [set([]) for i in range(4)]
for i in range(4):
    for X in skew_vector_fields:
        X_skew_monomial_basis[i] |= set(X[i].monomials())
X_skew_monomial_basis = [list(b) for b in X_skew_monomial_basis]
X_skew_monomial_index = [{m : k for k, m in enumerate(b)} for b in X_skew_monomial_basis]

X_skew_monomial_count = sum(len(b) for b in X_skew_monomial_basis)

X_skew_monomial_to_index = [{monomial : idx for (idx,monomial) in enumerate(X_skew_monomial_basis[j])} for j in range(4)] 
X_skew_evaluation_matrix = matrix(QQ, X_skew_monomial_count, len(skew_vector_fields), sparse=True)
for i in range(len(skew_vector_fields)):
    v = vector(QQ, X_skew_monomial_count, sparse=True)
    index_shift = 0
    for j in range(4):
        f = skew_vector_fields[i][j]
        for coeff, monomial in zip(f.coefficients(), f.monomials()):
            monomial_index = X_skew_monomial_to_index[j][monomial]
            v[index_shift + monomial_index] = coeff
        index_shift += len(X_skew_monomial_basis[j])
    X_skew_evaluation_matrix.set_column(i, v)

print('We have', X_skew_evaluation_matrix.rank(), 'linearly independent skewed vector fields obtained from the graphs. \n')

# We collect the linearly independent skewed vector fields and the corresponding encodings
pivots_2 = X_skew_evaluation_matrix.pivots()
print('A maximal subset of linearly independent skewed vector fields is given by graphs', list(pivots_2),'\n', )
independent_encodings_skew = [independent_encodings[k] for k in pivots_2]
skew_vector_fields_independent = [skew_vector_fields[k] for k in pivots_2]

# We now compute the tetrahedral flow. First, we create the Poisson bivector P and check that it satisfies [[P,P]]=0. 
P= (rho*epsilon).bracket(a1).bracket(a2)
if P.bracket(P)!=0:
    print('P is not a Poisson bivector. \n')

# Introduce the graph complex, and find the tetrahedron as a graph on 4 Poisson vertices and 6 edges in the cohomology. This 
# tetrahedron is unoriented, so we orient it. The next step is to make sure that the graph is built of wedges. This means 
# that there cannot be more than 2 outgoing edges at each vertex. This gives 2 graphs; one where 3 vertices have 2 outgoing
# edges, and one where 2 vertices have 2 outgoing edges and 2 vertices have 1 outgoing edge. 
# tetrahedron_oriented_filtered.show() gives a drawing of these graphs. 
# Finally, the bivector corresponding to the graph is computed.
GC=UndirectedGraphComplex(QQ, implementation='vector', sparse=True)
tetrahedron= GC.cohomology_basis(4,6)[0]
dGC=DirectedGraphComplex(QQ, implementation='vector')
tetrahedron_oriented= dGC(tetrahedron)
tetrahedron_oriented_filtered= tetrahedron_oriented.filter(max_out_degree=2)
#tetrahedron_oriented_filtered.show()
tetrahedron_operation= S4.graph_operation(tetrahedron_oriented_filtered)
Q_tetra= tetrahedron_operation(P, P, P, P) 
# !!! You don't want to print Q_tetra, it is very large
# print('The tetrahedral flow in 4D is', Q_tetra, '\n')

print('Q_tetra is computed.\n')

# Instead of solving the (very large) linear system directly, we use another method in 4D. This method is explained 
# in https://arxiv.org/abs/2112.03897. Essentially, we solve 2 (somewhat) smaller linear systems.
def casimir_flow(f):
    return 4*tetrahedron_operation(P,P,P,f)

a = [S4(a1), S4(a2)]
adot = [casimir_flow(a[0]), casimir_flow(a[1])]

X_a_multivectors = [[vector_field.bracket(casimir) for vector_field in skew_vector_fields_independent] for casimir in a]

X_a_basis = [set(flow_of_casimir[()].monomials()) for flow_of_casimir in adot]
for k in range(len(a)):
    for X_a_multivector in X_a_multivectors[k]:
        X_a_basis[k] |= set(X_a_multivector[()].monomials())
X_a_basis = [list(B) for B in X_a_basis]

X_a_monomial_index = [{m : k for k, m in enumerate(B)} for B in X_a_basis]
X_a_evaluation_matrix = [matrix(QQ, len(B), len(skew_vector_fields_independent), sparse=True) for B in X_a_basis]
for i in range(len(a)):
    for j in range(len(skew_vector_fields_independent)):
        v = vector(QQ, len(X_a_basis[i]), sparse=True)
        multivector = X_a_multivectors[i][j][()]
        for coeff, monomial in zip(multivector.coefficients(), multivector.monomials()):
            monomial_index = X_a_monomial_index[i][monomial]
            v[monomial_index] = coeff
        X_a_evaluation_matrix[i].set_column(j, v)

adot_vector = [vector(QQ, len(B)) for B in X_a_basis]
for i in range(len(a)):
    f = adot[i][()]
    for coeff, monomial in zip(f.coefficients(), f.monomials()):
        monomial_index = X_a_monomial_index[i][monomial]
        adot_vector[i][monomial_index] = coeff

P0 = (rho*epsilon).bracket(adot[0]).bracket(a2)
P1 = (rho*epsilon).bracket(a1).bracket(adot[1])
Q_remainder = Q_tetra - P0 - P1
P_without_rho = epsilon.bracket(a1).bracket(a2)
rhodot = Q_remainder[0,1] // P_without_rho[0,1]

X_rho_multivectors = [vector_field.bracket(rho*epsilon) for vector_field in skew_vector_fields_independent]

X_rho_basis = set(rhodot.monomials())
for X_rho_multivector in X_rho_multivectors:
    X_rho_basis |= set(X_rho_multivector[0,1,2,3].monomials())
X_rho_basis = list(X_rho_basis)

X_rho_monomial_index = {m : k for k, m in enumerate(X_rho_basis)}
X_rho_evaluation_matrix = matrix(QQ, len(X_rho_basis), len(skew_vector_fields_independent), sparse=True)
for j in range(len(skew_vector_fields_independent)):
    f = X_rho_multivectors[j][0,1,2,3]
    v = vector(QQ, len(X_rho_basis), sparse=True)
    for coeff, monomial in zip(f.coefficients(), f.monomials()):
        monomial_index = X_rho_monomial_index[monomial]
        v[monomial_index] = coeff
    X_rho_evaluation_matrix.set_column(j, v)

rhodot_vector = vector(QQ, len(X_rho_basis), sparse=True)
for coeff, monomial in zip(rhodot.coefficients(), rhodot.monomials()):
    monomial_index = X_rho_monomial_index[monomial]
    rhodot_vector[monomial_index] = coeff

big_matrix = X_a_evaluation_matrix[0].stack(X_a_evaluation_matrix[1]).stack(X_rho_evaluation_matrix)
big_vector = vector(list(-adot_vector[0]) + list(-adot_vector[1]) + list(-rhodot_vector))
X_solution_vector = big_matrix.solve_right(big_vector)
print('Proposition 14: There exist a solution over skewed vector fields from graphs. It is given by', X_solution_vector, '\n')

# Now, we create the evaluation matrix of the skewed independent vector fields; we need this in order to find a basis
# for the cocycle space. 
X_skew_ind_monomial_basis = [set([]) for i in range(4)]
for i in range(4):
    for vector_field in skew_vector_fields_independent:
        X_skew_ind_monomial_basis[i] |= set(vector_field[i].monomials())
X_skew_ind_monomial_basis = [list(b) for b in X_skew_ind_monomial_basis]
X_skew_ind_monomial_index = [{m : k for k, m in enumerate(b)} for b in X_skew_ind_monomial_basis]

X_skew_ind_monomial_count = sum(len(b) for b in X_skew_ind_monomial_basis)

X_skew_ind_evaluation_matrix = matrix(QQ, X_skew_ind_monomial_count, len(skew_vector_fields_independent), sparse=True)
for i in range(len(skew_vector_fields_independent)):
    v = vector(QQ, X_skew_ind_monomial_count, sparse=True)
    index_shift = 0
    for j in range(4):
        vector_field = skew_vector_fields_independent[i][j]
        for coeff, monomial in zip(vector_field.coefficients(), vector_field.monomials()):
            monomial_index = X_skew_ind_monomial_index[j][monomial]
            v[index_shift + monomial_index] = coeff
        index_shift += len(X_skew_ind_monomial_basis[j])
    X_skew_ind_evaluation_matrix.set_column(i, v)

# We compute a basis for the cocycle space.
X_cocycle_space = big_matrix.right_kernel().quotient(X_skew_ind_evaluation_matrix.right_kernel())
X_cocycles=[X_cocycle_space.lift(v) for v in X_cocycle_space.basis()]
print('Proposition 15: The cocycle space has dimension',X_cocycle_space.dimension(), '. The shifts are given by', X_cocycles, '\n')

# We compute the vector fields corresponding to the computed basis elements of the cocycle space.
shifts_formulas = [sum(X_cocycle[j]*skew_vector_fields_independent[j] for j in range(len(skew_vector_fields_independent))) for X_cocycle in X_cocycles]

# Now, we start the procedure of checking whether the shifts appearing in the cocycle space are Hamiltonian. 
hamiltonian_encodings= [(0, 1, 2, 4, 0, 1, 3, 5), (0, 1, 2, 4, 1, 2, 3, 5), (0, 1, 2, 4, 1, 3, 4, 5), (0, 2, 3, 4, 1, 2, 3, 5), (0, 2, 3, 4, 1, 3, 4, 5), (0, 2, 4, 5, 1, 3, 4, 5), (0, 1, 2, 4, 0, 2, 3, 5), (0, 1, 2, 4, 0, 3, 4, 5), (0, 1, 2, 4, 2, 3, 4, 5), (0, 2, 3, 4, 0, 2, 3, 5), (0, 2, 4, 5, 0, 2, 3, 5), (0, 2, 3, 4, 0, 3, 4, 5), (0, 2, 4, 5, 0, 3, 4, 5), (0, 2, 3, 4, 2, 3, 4, 5), (0, 2, 4, 5, 2, 3, 4, 5), (1, 2, 3, 4, 0, 2, 3, 5), (1, 2, 4, 5, 0, 3, 4, 5), (1, 2, 3, 4, 0, 3, 4, 5), (1, 2, 3, 4, 2, 3, 4, 5), (1, 2, 4, 5, 2, 3, 4, 5), (2, 3, 4, 5, 2, 3, 4, 5)]

# We find the graphs corresponding to the encodings. 
def hamiltonian_encoding_to_graph(encoding):
    targets = [encoding[0:4], encoding[4:8]]
    edges = sum([[(k+1,v) for v in t] for (k,t) in enumerate(targets)], [])
    return FormalityGraph(0, 6, edges)

hamiltonian_graphs = [encoding_to_graph(e) for e in hamiltonian_encodings]

# Since our vector fields are skew with respect to the Casimirs a1 and a2, we require that the Hamiltonians are symmetric with
# respect to a1 and a2; in this way, as the Poisson bivector P itself is skew (wrt a1 and a2), we guarantee that the 
# Hamiltonian vector  fields are also skew symmetric wrt a1 and a2.  

hamiltonian_encodings_interchanged= [(0, 1, 4, 2, 0, 1, 5, 3), (0, 1, 4, 2, 1, 4, 5, 3), (0, 1, 4, 2, 1, 5, 2, 3), (0, 4, 5, 2, 1, 4, 5, 3), (0, 4, 5, 2, 1, 5, 2, 3), (0, 4, 2, 3, 1, 5, 2, 3), (0, 1, 4, 2, 0, 4, 5, 3), (0, 1, 4, 2, 0, 5, 2, 3), (0, 1, 4, 2, 4, 5, 2, 3), (0, 4, 5, 2, 0, 4, 5, 3), (0, 4, 2, 3, 0, 4, 5, 3), (0, 4, 5, 2, 0, 5, 2, 3), (0, 4, 2, 3, 0, 5, 2, 3), (0, 4, 5, 2, 4, 5, 2, 3), (0, 4, 2, 3, 4, 5, 2, 3), (1, 4, 5, 2, 0, 4, 5, 3), (1, 4, 2, 3, 0, 5, 2, 3), (1, 4, 5, 2, 0, 5, 2, 3), (1, 4, 5, 2, 4, 5, 2, 3), (1, 4, 2, 3, 4, 5, 2, 3), (4, 5, 2, 3, 4, 5, 2, 3)]

hamiltonian_graphs_interchanged = [encoding_to_graph(e) for e in hamiltonian_encodings_interchanged]

# We create a new function to evaluate the Hamiltonian graphs to Hamiltonian formulas. 
def evaluate_ham(g): 
    result = S4.zero()
    for index_choice in itertools.product(itertools.permutations(range(4)), repeat=2):
        sign = epsilon[index_choice[0]] * epsilon[index_choice[1]]
        vertex_content = [S4(rho), S4(rho), S4(a1), S4(a1), S4(a2), S4(a2)]
        for ((source, target), index) in zip(g.edges(), sum(map(list, index_choice), [])):
            vertex_content[target] = vertex_content[target].derivative(even_coords[index])
        result += sign * prod(vertex_content)
    return result

# We compute the Hamiltonian formulas.
hamiltonian_formulas = []
with Pool(processes=24) as pool:
    hamiltonian_formulas = list(pool.imap(evaluate_ham, hamiltonian_graphs))

# We compute the Hamiltonian formulas with a1 and a2 swapped.
hamiltonian_formulas_interchanged = []
with Pool(processes=24) as pool:
    hamiltonian_formulas_interchanged = list(pool.imap(evaluate_ham, hamiltonian_graphs_interchanged))

# We obtain the symmetrized Hamiltonian formulas.
hamiltonian_formulas_symm=[]
for i in range(len(hamiltonian_formulas)):
    hamiltonian_formulas_symm.append(1/2*(hamiltonian_formulas[i]+hamiltonian_formulas_interchanged[i]))

# We check if we have symmetrized Hamiltonian formulas that evaluate to 0. 
print('We have', hamiltonian_formulas_symm.count(0), 'Hamiltonian graph that evaluates to 0.')
print('The Hamiltonian graph evaluated to 0 is:\n')
for (k,ham) in enumerate(hamiltonian_formulas_symm):
    if ham==0:
        print('Hamiltonian', k+1,'\n')

#From the symmetrized Hamiltonians we compute the skew-symmetrized Hamiltonian vector fields.
hamiltonian_vector_fields_skew=[]
for formula in hamiltonian_formulas_symm:
    hamiltonian_vector_fields_skew.append(P.bracket(formula))

# We create the Hamiltonian monomial basis to be able to express the basis of the cocycle space in terms of the monomials
# appearing in the skewed Hamiltonian vector fields. 
hamiltonian_monomial_basis = {}
for j in range(len(shifts_formulas)):
    for i in range(4): 
        hamiltonian_monomial_basis[i] = set(shifts_formulas[j][i].monomials())
        for formula in hamiltonian_vector_fields_skew:
            hamiltonian_monomial_basis[i] |= set(formula[i].monomials())

hamiltonian_monomial_basis = {idx: list(b) for idx, b in hamiltonian_monomial_basis.items()}
hamiltonian_monomial_index = {idx: {m : k for k, m in enumerate(b)} for idx, b in hamiltonian_monomial_basis.items()}
hamiltonian_monomial_count = sum(len(b) for b in hamiltonian_monomial_basis.values())

def shift_formula_to_vector(shift_formula):
        shift_vector = vector(QQ,hamiltonian_monomial_count, sparse=True)
        index_offset = 0
        for i in hamiltonian_monomial_basis:
            for coeff, monomial in shift_formula[i]:
                monomial_index = hamiltonian_monomial_index[i][monomial]
                shift_vector[monomial_index + index_offset] = coeff
            index_offset += len(hamiltonian_monomial_basis[i])
        return shift_vector

shifts_vectors = [shift_formula_to_vector(shift_formula) for shift_formula in shifts_formulas]

# Let us collect which monomials appear in which skewed Hamiltonians vector fields, and store this in an evaluation matrix. 
hamiltonian_evaluation_matrix = matrix(QQ,hamiltonian_monomial_count,len(hamiltonian_vector_fields_skew),sparse=True)
for k in range(len(hamiltonian_vector_fields_skew)):
    vector_field = hamiltonian_vector_fields_skew[k] 
    v = vector(QQ, hamiltonian_monomial_count, sparse=True) 
    index_shift = 0
    for i in hamiltonian_monomial_basis: 
        for coeff, monomial in vector_field[i]:
            monomial_index = hamiltonian_monomial_index[i][monomial]
            v[monomial_index +index_shift] = coeff
        index_shift += len(hamiltonian_monomial_basis[i])
    hamiltonian_evaluation_matrix.set_column(k, v)

# Let us check the nullity of the skewed Hamiltonian vector fields among themselves.
nullity=hamiltonian_evaluation_matrix.right_nullity()
kernel_basis= hamiltonian_evaluation_matrix.right_kernel().basis()
print('The nullity among the skewed Hamiltonian vector fields is ', nullity, '.\n')
# Note that the below is on the level of vector fields, not formulas. Unfortunately, the evaluation_matrix procedure only
# works from vector fields onward, and not for 0-vectors (formulas). You can explictly check that the linear relations ARE 
# already preserved on the level of formulas by simply evaluating hamiltonian_formulas[x]==hamiltonian_formulas[y] etc.
print('Explicitly, these are the linear combinations that evaluate to 0:', kernel_basis, '\n')

# We can now solve the shifts! Note that if (at least one) shift is NOT Hamiltonian, the code will break here as no solution 
# can be found.
shifts_solutions = [hamiltonian_evaluation_matrix.solve_right(shift_vector) for shift_vector in shifts_vectors]


# Write the solutions above from combinations of graphs to their vector field forms to check validity.
solution_formulas = [sum(shift_solution[i]*hamiltonian_vector_fields_skew[i] for i in range(len(shift_solution))) for shift_solution in shifts_solutions]


# We check that the solutions we found in terms of the skewed Hamiltonian vector fields are truly correct; we check 
# explicitly whether they agree on the previously computed formulass for the basis elements of the cocycle space, and
# that they ARE elements of the cocycle space (i.e. they evaluate to 0 under [[P, ]]).  
if any(P.bracket(solution_formula) != 0 for solution_formula in solution_formulas) or solution_formulas != shifts_formulas:
    print('There is an error in computing the shifts.')
else:
    print('Theorem 17: The shifts in the cocycle space are Hamiltonian.')

for k, shift_solution in enumerate(shifts_solutions):
        print('The shift #', k+1, 'is the following linear combination of Hamiltonian vector fields:', shift_solution)

We have 324 graphs.

There are 54 graphs that evaluate to 0 under the morhpism from graphs to multivectors.
These graphs are:
graph 32
graph 38
graph 63
graph 75
graph 85
graph 88
graph 112
graph 113
graph 115
graph 119
graph 139
graph 142
graph 144
graph 156
graph 166
graph 167
graph 169
graph 172
graph 173
graph 174
graph 181
graph 182
graph 183
graph 193
graph 196
graph 202
graph 203
graph 204
graph 220
graph 223
graph 233
graph 234
graph 239
graph 240
graph 247
graph 250
graph 252
graph 253
graph 254
graph 255
graph 262
graph 263
graph 264
graph 274
graph 277
graph 287
graph 288
graph 293
graph 294
graph 301
graph 304
graph 322
graph 323
graph 324

The vector fields have 201 linear relations among themselves.

 A maximal subset of linearly independent graphs is given by: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 22, 23, 24, 25, 26, 28, 29, 32, 34, 35, 38, 40, 41, 43, 44, 47, 50, 52, 53, 56, 59, 68, 71, 80, 81, 82, 83, 93, 94, 95, 96, 97, 102, 104, 105, 106